In [ ]:
import random
from typing import Dict

from google.adk.agents import Agent
from google.adk.tools import FunctionTool



# GAME STATE

VALID_MOVES = ["rock", "paper", "scissors", "bomb"]

game_state = {
    "round": 0,
    "max_rounds": 3,
    "user_score": 0,
    "bot_score": 0,
    "user_bomb_used": False,
    "bot_bomb_used": False,
    "history": []
}



# TOOLS


def validate_move(player: str, move: str, state: Dict) -> Dict:
    """Validate a move and enforce bomb usage rules."""
    move = move.lower().strip()

    if move not in VALID_MOVES:
        return {"valid": False, "reason": "Invalid move", "penalty": None}

    if move == "bomb":
        if player == "user" and state["user_bomb_used"]:
            return {
                "valid": False,
                "reason": "User bomb already used",
                "penalty": "bot"
            }
        if player == "bot" and state["bot_bomb_used"]:
            return {
                "valid": False,
                "reason": "Bot bomb already used",
                "penalty": "user"
            }

    return {"valid": True, "move": move}


def resolve_round(user_move: str, bot_move: str) -> Dict:
    """Determine round winner."""
    if user_move == bot_move:
        return {"winner": "draw"}

    if user_move == "bomb" and bot_move == "bomb":
        return {"winner": "draw"}

    if user_move == "bomb":
        return {"winner": "user"}

    if bot_move == "bomb":
        return {"winner": "bot"}

    beats = {
        "rock": "scissors",
        "scissors": "paper",
        "paper": "rock"
    }

    return {"winner": "user" if beats[user_move] == bot_move else "bot"}


def update_game_state(state: Dict, user_move: str, bot_move: str, winner: str) -> Dict:
    """Persist game state."""
    state["round"] += 1

    if user_move == "bomb":
        state["user_bomb_used"] = True
    if bot_move == "bomb":
        state["bot_bomb_used"] = True

    if winner == "user":
        state["user_score"] += 1
    elif winner == "bot":
        state["bot_score"] += 1

    state["history"].append({
        "round": state["round"],
        "user_move": user_move,
        "bot_move": bot_move,
        "winner": winner
    })

    return state



# TOOL WRAPPERS (ADK)


validate_move_tool = FunctionTool(validate_move)
resolve_round_tool = FunctionTool(resolve_round)
update_game_state_tool = FunctionTool(update_game_state)



# CUSTOM AGENT


class RPSRefereeAgent(Agent):
    """
    Custom ADK agent that orchestrates the game.
    """

    def play_round(self, user_input: str) -> str:
        validation = validate_move("user", user_input, game_state)

        # Invalid move
        if not validation["valid"]:
            game_state["round"] += 1

            # Penalize if bomb reused
            if validation.get("penalty") == "bot":
                game_state["bot_score"] += 1
                return (
                    f"Invalid move → {validation['reason']}\n"
                    f"Bot gets the point (illegal bomb use)."
                )

            return f"Invalid move → {validation['reason']} (round forfeited)"

        user_move = validation["move"]
        bot_move = self.bot_choose_move()

        result = resolve_round(user_move, bot_move)
        winner = result["winner"]

        update_game_state(game_state, user_move, bot_move, winner)

        return (
            f"You played: {user_move}\n"
            f"Bot played: {bot_move}\n"
            f"Round winner: {winner.upper()}"
        )

    def bot_choose_move(self) -> str:
        moves = ["rock", "paper", "scissors"]
        if not game_state["bot_bomb_used"]:
            moves.append("bomb")
        return random.choice(moves)


# AGENT INSTANTIATION

referee_agent = RPSRefereeAgent(
    name="rps_referee",
    tools=[
        validate_move_tool,
        resolve_round_tool,
        update_game_state_tool
    ]
)



# CLI LOOP


def print_final_result():
    print("\n====================")
    print("GAME OVER")
    print("====================")
    print(f"Final Score → You: {game_state['user_score']} | Bot: {game_state['bot_score']}")

    if game_state["user_score"] > game_state["bot_score"]:
        print("You win!")
    elif game_state["bot_score"] > game_state["user_score"]:
        print("Bot wins!")
    else:
        print("Draw!")


def run_game():
    print("\nWelcome to Rock–Paper–Scissors–Plus!")
    print("• Best of 3 rounds")
    print("• Moves: rock, paper, scissors, bomb (once)")
    print("• Bomb beats all; bomb vs bomb = draw")
    print("• Illegal bomb reuse gives opponent a point")

    while game_state["round"] < game_state["max_rounds"]:
        print(f"\n--- Round {game_state['round'] + 1} ---")
        user_input = input("Your move: ")

        response = referee_agent.play_round(user_input)
        print(response)

    print_final_result()


if __name__ == "__main__":
    run_game()
